In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
import numpy as np
import os
import psycopg2
from dotenv import load_dotenv

from models import Player, GameStats

In [2]:
load_dotenv()
DB_USERNAME = os.environ['DB_USERNAME']
DB_PASSWORD = os.environ['DB_PASSWORD']
DB_HOST = os.environ['DB_HOST']
DB_PORT = os.environ['DB_PORT']
DB_NAME = os.environ['DB_NAME']

DB_URL = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DB_URL, echo=False)

In [3]:
Session = sessionmaker(bind=engine)
session = Session()
player_df = pd.read_csv('players.csv')
player_df['birth_date'] = pd.to_datetime(player_df['birth_date'], errors='coerce')
player_df.replace({np.nan : None}, inplace=True)


In [4]:
# upload player data

no_errors = True
try:
    for id, player_data in player_df.iterrows():
        existing_or_new_player = Player(
            id=id,
            gsis_id = player_data['gsis_id'],
            gsis_it_id = player_data['gsis_it_id'],
            ktc_id = None,

            status = player_data['status'],
            display_name = player_data['display_name'],
            football_name = player_data['football_name'],
            birth_date = player_data['birth_date'],

            position_group = player_data['position_group'],
            position = player_data['position'],

            height = player_data['height'],
            weight = player_data['weight'],
            yoe = player_data['years_of_experience'],
            team_abbr = player_data['team_abbr'],
            current_team_id = player_data['current_team_id'],

            entry_year = player_data['entry_year'],
            rookie_year= player_data['entry_year'],

            college = player_data['college_name'],
            college_conf = player_data['college_conference'],
            draft_club = player_data['draft_club'],
            draft_number = player_data['draft_number'],
            draft_round = player_data['draft_round'],

            uniform_number = player_data['uniform_number'],
            jersey_number = player_data['jersey_number']
        )
        session.merge(existing_or_new_player)
except psycopg2.DataError as e:
    print('\n ----------------- Error ---------------')
    # print(e)
    print(player_data)
    no_errors = False
    session.rollback()
    
if no_errors:
    session.commit()
    n_player = session.query(Player).count()
    print(f'\n {n_player} Players added or updated.')
    


 20039 Players added or updated.


In [5]:
game_df = pd.read_csv('player_stats.csv')
game_df.replace({np.nan : None}, inplace=True)

In [6]:
game_df.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [7]:
no_errors = True
try:
    for id, game_data in game_df.iterrows():
        player = session.query(Player).filter_by(gsis_id=game_data['player_id']).first()
        
        existing_or_new_player = GameStats(
            id = id,
            player = player,
            season = game_data['season'],
            week = game_data['week'],
            season_type = game_data['season_type'],
            opponent_team = game_data['opponent_team'],

            completions = game_data['completions'],
            attempts = game_data['attempts'],
            passing_yards = game_data['passing_yards'],
            passing_tds = game_data['passing_tds'],
            interceptions = game_data['interceptions'],
            sacks = game_data['sacks'],
            sack_yards = game_data['sack_yards'],
            sack_fumbles = game_data['sack_fumbles'],
            sack_fumbles_lost = game_data['sack_fumbles_lost'],
            passing_air_yards = game_data['passing_air_yards'],
            passing_yards_after_catch = game_data['passing_yards_after_catch'],
            passing_first_downs = game_data['passing_first_downs'],
            passing_epa = game_data['passing_epa'],

            rushing_tds = game_data['rushing_tds'],
            rushing_fumbles = game_data['rushing_fumbles'],
            rushing_fumbles_lost = game_data['rushing_fumbles_lost'],
            rushing_first_downs = game_data['rushing_first_downs'],
            rushing_epa = game_data['rushing_epa'],
            rushing_2pt_conversion = game_data['rushing_2pt_conversions'],

            receptions = game_data['receptions'],
            targets = game_data['targets'],
            receiving_yards = game_data['receiving_yards'],
            receiving_tds = game_data['receiving_tds'],
            receiving_fumbles = game_data['receiving_fumbles'],
            receiving_air_yards = game_data['receiving_air_yards'],
            receiving_yards_after_catch = game_data['receiving_yards_after_catch'],
            receiving_first_downs = game_data['receiving_first_downs'],
            receiving_epa = game_data['receiving_epa'],
            racr = game_data['racr'],
            target_share = game_data['target_share'],
            air_yards_share = game_data['air_yards_share'],
            wopr = game_data['wopr'],
            special_teams_tds = game_data['special_teams_tds'],

            fantasy_points = game_data['fantasy_points'],
            fantasy_points_ppr = game_data['fantasy_points_ppr']
        )

        session.merge(existing_or_new_player)
except psycopg2.DataError as e:
    print('\n ----------------- Error ---------------')
    # print(e)
    print(player_data)
    no_errors = False
    session.rollback()
    
if no_errors:
    session.commit()
    n_player = session.query(GameStats).count()
    print(f'\n {n_player} Games added or updated.')
    

C:\Users\simer\AppData\Local\Temp\ipykernel_11940\4127028382.py:54: SAWarning: Object of type <GameStats> not in session, add operation along 'Player.game_stats' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  session.merge(existing_or_new_player)



 129741 Games added or updated.


In [8]:
session.close()